In [1]:
from random import random
from time import sleep
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import as_completed


## Map and Wait

Perhaps the most common pattern when using the `ThreadPoolExecutor` is to convert a for loop that executes a function on each item in a collection to use threads.

It assumes that the function has no side effects, meaning it does not access any data outside of the function and does not change the data provided to it. It takes data and produces a result.

Each application of the function to an item in the collection is a task that is executed asynchronously.

In [2]:
def task(value1, value2):
    # Sleep for less than a second
    sleep(random())
    return (value1, value2)


# Start the thread pool
with ThreadPoolExecutor() as executor:
    # Submit all tasks
    for result in executor.map(task, ['1', '2', '3'], ['a', 'b', 'c']):
        print(result)


('1', 'a')
('2', 'b')
('3', 'c')


## Submit and Use As Completed

The second most common pattern when using the `ThreadPoolExecutor` is to submit tasks and use the results **as they become available**.

This can be achieved using the `submit()` function to push tasks into the thread pool that returns `Future` objects, then calling the module method `as_completed()` on the list of Future objects that will return each `Future` object as its task is completed.

The function `as_completed()` returns `Future` objects from a collection as they complete their execution **in whatever order**.

The example below demonstrates this pattern, submitting the tasks in order from 0 to 9 and showing results in the order that they were completed.

In [3]:
def task(name):
    # Sleep for less than a second
    sleep(random())
    return name
 
# Start the thread pool
with ThreadPoolExecutor(10) as executor:
    # Submit tasks and collect futures
    futures = [executor.submit(task, i) for i in range(10)]
    # Process task results as they are available
    for future in as_completed(futures):
        # Retrieve the result
        print(future.result())


9
1
6
7
8
4
3
2
0
5


This is different from iterating over the results from calling `map()` in two ways. 

* First, `map()` returns an iterator over objects, not over `Future` objects. 
  
* Second, `map()` returns results **in the order that the tasks were submitted**, not in the order that they are completed.

## Submit and Use Sequentially

We may require the results from tasks in the order that the tasks were submitted, e.g., the tasks have a natural ordering.

We can implement this pattern by calling `submit()` for each task to obtain a list of `Future` objects, then iterating over the `Future` objects **in the order** that the tasks were submitted and retrieving the results.

The main difference between this pattern and the "as completed" pattern is that we enumerate the list of futures directly, instead of calling the `as_completed()` function.

In [4]:
def task(name):
    sleep(random())
    return name
 
with ThreadPoolExecutor(10) as executor:
    # Submit tasks and collect futures
    futures = [executor.submit(task, i) for i in range(10)]
    # Process task results in the order they were submitted
    for future in futures:
        print(future.result())


0
1
2
3
4
5
6
7
8
9


## Submit and Use Callback

We may not want to explicitly process the results once they are available; instead, we want to call a function on the result.

Instead of doing this manually, such as in the as completed pattern above, we can have the thread pool call the function for us with the result automatically.

This can be achieved by setting a `callback` on each `Future` object by calling the `add_done_callback()` function and passing the name of the function.

The thread pool will then call the callback function as each task completes, passing in `Future` objects for the task.


In [5]:
def task(name):
    sleep(random())
    return name
 
# Custom callback function called on tasks when they complete
def custom_callback(future):
    print('The future object', future.result(), 'is acted upon')
    
def custom_callback2(future):
    print('The future object', future.result(), 'is acted upon, again')
 
with ThreadPoolExecutor(10) as executor:
    # Submit tasks and collect futures
    futures = [executor.submit(task, i) for i in range(10)]
    # Register the callback on all tasks
    for future in futures:
        future.add_done_callback(custom_callback)
        # Second callback
        future.add_done_callback(custom_callback2)


The future object 5 is acted upon
The future object 5 is acted upon, again
The future object 6 is acted upon
The future object 6 is acted upon, again
The future object 8 is acted upon
The future object 8 is acted upon, again
The future object 7 is acted upon
The future object 7 is acted upon, again
The future object 1 is acted upon
The future object 1 is acted upon, again
The future object 3 is acted upon
The future object 3 is acted upon, again
The future object 2 is acted upon
The future object 2 is acted upon, again
The future object 4 is acted upon
The future object 4 is acted upon, again
The future object 9 is acted upon
The future object 9 is acted upon, again
The future object 0 is acted upon
The future object 0 is acted upon, again


Notice that the returned results is no longer in the order that they were submitted. In addition, we can also see that the two callback functions are called for each task in the order that we registered them with each Future object.

## Submit and Wait for All

It is common to submit all tasks and then wait for all tasks in the thread pool to complete.

This pattern may be useful when tasks do not return a result directly, such as if each task stores the result in a resource directly such as a file.



In [6]:
def task(name):
    sleep(random())
    # Display the result, rather than return
    print(name)
 
with ThreadPoolExecutor(10) as executor:
    # submit tasks and collect futures
    futures = [executor.submit(task, i) for i in range(10)]
    # wait for all tasks to complete
print('All tasks are done!')


96

2
7
1
3
5
0
8
4
All tasks are done!


The main thread does not move on and print the message until all tasks are completed, after the thread pool has been automatically shut down by the context manager.

## Submit and Wait for First

It is common to issue many tasks and only be concerned with the first result returned.

That is, not the result of the first task, but a result from any task that happens to be the first to complete its execution.

This may be the case if we are trying to access the same resource from multiple locations, like a file or some data.

This pattern can be achieved using the `wait()` module function and setting the `return_when` argument to the `FIRST_COMPLETED` constant.

In [7]:
from concurrent.futures import FIRST_COMPLETED, wait

def task(name):
    sleep(random())
    return name
 
# Start the thread pool
executor = ThreadPoolExecutor(10)
# Submit tasks and collect futures
futures = [executor.submit(task, i) for i in range(10)]

# Wait until any task completes first
done, not_done = wait(futures, return_when=FIRST_COMPLETED)
# Fet the result from the first task to complete
print(done.pop().result())
# Shutdown without waiting
executor.shutdown(wait=False, cancel_futures=True)


3


Running the example will wait for any of the tasks to complete, then retrieve the result of the first completed task and shut down the thread pool.

Importantly, the tasks will continue to execute in the thread pool in the background and the main thread will not close until all tasks have completed.